In [2]:
import re
import torch
import numpy as np
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data.sampler import SubsetRandomSampler
from matplotlib import pyplot as plt
import yaml
import functools

In [3]:
from generator import Generator
set_file = "sets.yml"
with open(set_file, 'r') as outfile:
    setf = yaml.safe_load(outfile)
sets = setf[0]+setf[1]+setf[2]
print(sets)


['A', 'N', 'Q', 'G', 'O', 'I', 'L', 'M', 'F', 'P', 'U', 'S', 'T', 'W', 'V', 'D', 'C', 'E', 'Y', 'R', 'H', 'K']


In [18]:
gen = Generator()
for k in tqdm( range(20000)):
    combine = []
    coms = []
    com = ''
    tokens_pc = ''
    datainput = ''
    rand_len = np.random.randint(2,5)
    for i in range(0, rand_len):
        rand_idx = np.random.randint(len(sets))
        combine.append(sets[rand_idx])
        com = com+sets[rand_idx]+' '
    combine.append('<EOS>')
    if com in coms :
        continue
    coms.append(com)
    tokens_pot, tokens_cap = gen.get(combine)
    #print(tokens_pot, tokens_cap)
    if len(tokens_cap) != 0:
        tokens_cap = tokens_cap[:-1]
    for i in tokens_pot:
        tokens_pc = tokens_pc + str(i) +' '
    for i in tokens_cap:
        tokens_pc = tokens_pc + str(i) +' '

    com= com[:-1]
    tokens_pc = tokens_pc[:-1]

    datainput = com + '\t' + tokens_pc + '\n'
    

    #print(datainput)

    with open('bigdata.txt', 'a') as file:
        file.write(datainput)

SyntaxError: invalid syntax (1195659559.py, line 15)

In [3]:
with open('mydata.txt') as f:
    sentences = f.readlines()
# print(sentences)

In [4]:
NUM_INSTANCES = 10000
MAX_SENT_LEN = 10
eng_sentences, deu_sentences = [], []
eng_words, deu_words = set(), set()
for i in tqdm(range(NUM_INSTANCES)):
  rand_idx = np.random.randint(len(sentences))
  # find only letters in sentences
  eng_sent, deu_sent = ["<sos>"], ["<sos>"]

  eng_sent += re.findall(r"\w+", sentences[rand_idx].split("\t")[1]) 
  deu_sent += re.findall(r"\w+", sentences[rand_idx].split("\t")[0])

  # change to lowercase
  eng_sent = [x.lower() for x in eng_sent]
  deu_sent = [x.lower() for x in deu_sent]
  eng_sent.append("<eos>")
  deu_sent.append("<eos>")

  if len(eng_sent) >= MAX_SENT_LEN:
    eng_sent = eng_sent[:MAX_SENT_LEN]
  else:
    for _ in range(MAX_SENT_LEN - len(eng_sent)):
      eng_sent.append("<pad>")

  if len(deu_sent) >= MAX_SENT_LEN:
    deu_sent = deu_sent[:MAX_SENT_LEN]
  else:
    for _ in range(MAX_SENT_LEN - len(deu_sent)):
      deu_sent.append("<pad>")

  # add parsed sentences
  eng_sentences.append(eng_sent)
  deu_sentences.append(deu_sent)

  # update unique words
  eng_words.update(eng_sent)
  deu_words.update(deu_sent)

eng_words, deu_words = list(eng_words), list(deu_words)

# encode each token into index
for i in tqdm(range(len(eng_sentences))):
  eng_sentences[i] = [eng_words.index(x) for x in eng_sentences[i]]
  deu_sentences[i] = [deu_words.index(x) for x in deu_sentences[i]]

idx = 10
print(eng_sentences[idx])
print([eng_words[x] for x in eng_sentences[idx]])
print(deu_sentences[idx])
print([deu_words[x] for x in deu_sentences[idx]])

100%|██████████| 10000/10000 [00:00<00:00, 30506.85it/s]

[235, 22, 107, 413, 404, 441, 28, 423, 296, 296]
['<sos>', '362', '622', '881', '0', '331', '912', '<eos>', '<pad>', '<pad>']
[20, 5, 16, 4, 9, 15, 15, 15, 15, 15]
['<sos>', 'w', 'd', 'n', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


In [5]:
ENG_VOCAB_SIZE = len(eng_words)
DEU_VOCAB_SIZE = len(deu_words)
NUM_EPOCHS = 20
HIDDEN_SIZE = 16
EMBEDDING_DIM = 30
BATCH_SIZE = 128
NUM_HEADS = 2
NUM_LAYERS = 3
LEARNING_RATE = 1e-3
DROPOUT = .3
DEVICE = torch.device('cpu') 

In [6]:
def get_output_mask(inp, pad_idx=1):
    return torch.triu(inp.new_ones(inp.size(1),inp.size(1)), diagonal=1)[None,None].byte()

def compose(*functions):
    return functools.reduce(lambda f, g: lambda x: f(g(x)), functions, lambda x: x)

In [7]:
class MTDataset(torch.utils.data.Dataset):
  def __init__(self):
    # import and initialize dataset    
    self.source = np.array(eng_sentences, dtype = int)
    self.target = np.array(deu_sentences, dtype = int)
    
  def __getitem__(self, idx):
    # get item by index
    return self.source[idx], self.target[idx]
  
  def __len__(self):
    # returns length of data
    return len(self.source)

np.random.seed(777)   # for reproducibility
dataset = MTDataset()
NUM_INSTANCES = len(dataset)
TEST_RATIO = 0.3
TEST_SIZE = int(NUM_INSTANCES * 0.3)

indices = list(range(NUM_INSTANCES))

test_idx = np.random.choice(indices, size = TEST_SIZE, replace = False)
train_idx = list(set(indices) - set(test_idx))
train_sampler, test_sampler = SubsetRandomSampler(train_idx), SubsetRandomSampler(test_idx)

train_loader = torch.utils.data.DataLoader(dataset, batch_size = BATCH_SIZE, sampler = train_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size = BATCH_SIZE, sampler = test_sampler)

In [8]:
## source: https://pytorch.org/tutorials/beginner/transformer_tutorial.html
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [9]:
class TransformerNet(nn.Module):
  def __init__(self, num_src_vocab, num_tgt_vocab, embedding_dim, hidden_size, nheads, n_layers, max_src_len, max_tgt_len, dropout):
    super(TransformerNet, self).__init__()
    # embedding layers
    self.enc_embedding = nn.Embedding(num_src_vocab, embedding_dim)
    self.dec_embedding = nn.Embedding(num_tgt_vocab, embedding_dim)

    # positional encoding layers
    self.enc_pe = PositionalEncoding(embedding_dim, max_len = max_src_len)
    self.dec_pe = PositionalEncoding(embedding_dim, max_len = max_tgt_len)

    # encoder/decoder layers
    enc_layer = nn.TransformerEncoderLayer(embedding_dim, nheads, hidden_size, dropout)
    dec_layer = nn.TransformerDecoderLayer(embedding_dim, nheads, hidden_size, dropout)
    self.encoder = nn.TransformerEncoder(enc_layer, num_layers = n_layers)
    self.decoder = nn.TransformerDecoder(dec_layer, num_layers = n_layers)

    # final dense layer
    self.dense = nn.Linear(embedding_dim, num_tgt_vocab)
    self.log_softmax = nn.LogSoftmax()

  def forward(self, src, tgt):
    mask_tgt = get_output_mask(tgt)
    src, tgt = self.enc_embedding(src).permute(1, 0, 2), self.dec_embedding(tgt).permute(1, 0, 2)
    src, tgt = self.enc_pe(src), self.dec_pe(tgt)
    memory = compose(self.encoder)(src)
    transformer_out = compose(self.decoder)(tgt, memory, mask_tgt)
    final_out = self.dense(transformer_out)
    return self.log_softmax(final_out)

In [10]:
model = TransformerNet(ENG_VOCAB_SIZE, DEU_VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_SIZE, NUM_HEADS, NUM_LAYERS, MAX_SENT_LEN, MAX_SENT_LEN, DROPOUT).to(DEVICE)
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

/home/postgrads/2650807G/spack/opt/spack/linux-centos7-x86_64_v3/gcc-4.8.5/anaconda3-2022.10-ldbv2ghkumdvkfs6y6xxwrtticnzqxhf/envs/BioTokens-cpu/lib/python3.11/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [11]:
%%time
loss_trace = []
for epoch in tqdm(range(NUM_EPOCHS)):
  current_loss = 0
  for i, (x, y) in enumerate(train_loader):
    x, y  = x.to(DEVICE), y.to(DEVICE)
    outputs = model(x, y)
    loss = criterion(outputs.permute(1, 2, 0), y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    current_loss += loss.item()
  loss_trace.append(current_loss)
torch.save(model, 'mymodel.pt')

# loss curve
plt.plot(range(1, NUM_EPOCHS+1), loss_trace, 'r-')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()


  0%|          | 0/20 [00:00<?, ?it/s]


RuntimeError: value cannot be converted to type uint8_t without overflow

In [ ]:
correct, total = 0, 0
predictions = []
lmatch = 0
model.eval()
for i, (x, y) in enumerate(test_loader):
    with torch.no_grad():
        x, y  = x.to(DEVICE), y.to(DEVICE)
        outputs = model(x,y)
        _, y_pred = torch.max(outputs.permute(1,2,0).data, 1)
        for j in range(y.shape[0]):
            input_x =  [eng_words[k] for k in x[j]]
            target_y = [deu_words[k] for k in y[j]]
            output_y = [deu_words[k] for k in y_pred[j]]
            print(input_x)
            print(target_y)
            print(output_y)
            if len(target_y) == len(output_y):
                for u in range(len(target_y)):
                    if target_y[u] == '<eos>':
                        break
                    total += 1
                    if target_y[u] == output_y[u]:
                        correct += 1
            else:
                lmatch += 1

print(correct/total) 
print(lmatch) 


In [ ]:
correct, total = 0, 0
predictions = []
for i, (x,y) in enumerate(test_loader):
  with torch.no_grad():
    x, y  = x.to(DEVICE), y.to(DEVICE)
    outputs = model(x,y)
    _, y_pred = torch.max(outputs.permute(1,2,0).data, 1)
    print(y[0])
    print(y_pred[0])
    total += y.shape[0]*y.shape[1]
    correct += (y_pred == y).sum().item()

print(correct/total)

In [ ]:
model_scripted = torch.jit.script(model) # Export to TorchScript
model_scripted.save('mymodel.pt')